In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import autosklearn.classification
from sklearn.model_selection import train_test_split
from scipy import sparse
from sklearn.preprocessing import OneHotEncoder

/usr/lib/python3/dist-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)


In [ ]:
data = pd.read_csv('game_data_public.KHM.PremierDraft.csv')

In [31]:
decks = data.loc[:,data.columns.str.contains("deck")]
decks = decks.drop(columns=['deck_Forest', 'deck_Island', 'deck_Mountain', 'deck_Plains',
       'deck_Swamp','deck_Snow-Covered Forest', 'deck_Snow-Covered Island',
       'deck_Snow-Covered Mountain','deck_Snow-Covered Swamp','deck_Snow-Covered Plains'])
decks_long = decks
decks.columns

Index(['deck_Absorb Identity', 'deck_Aegar, the Freezing Flame',
       'deck_Alpine Meadow', 'deck_Alrund's Epiphany',
       'deck_Alrund, God of the Cosmos', 'deck_Annul', 'deck_Arachnoform',
       'deck_Arctic Treeline', 'deck_Armed and Armored',
       'deck_Arni Brokenbrow',
       ...
       'deck_Vorinclex, Monstrous Raider', 'deck_Waking the Trolls',
       'deck_Warchanter Skald', 'deck_Warhorn Blast',
       'deck_Weathered Runestone', 'deck_Weigh Down',
       'deck_Wings of the Cosmos', 'deck_Withercrown', 'deck_Woodland Chasm',
       'deck_Youthful Valkyrie'],
      dtype='object', length=311)

In [7]:
cls = autosklearn.classification.AutoSklearnClassifier()


In [ ]:
DATA_SIZE=100
X = decks.loc[:DATA_SIZE]
y = data.loc[:DATA_SIZE,"won"].to_numpy().astype(bool)
X = np.array(X>0, dtype=short)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
cls.fit(X_train, y_train)